# Import datasets taken from MOH

In [1]:
import pandas as pd

In [2]:
cases_df = pd.read_csv("cases_state.csv")
cases_df.head()

,date,state,cases_new,cases_import,cases_recovered,cases_active,cases_cluster,cases_unvax,cases_pvax,cases_fvax,...,cases_0_4,cases_5_11,cases_12_17,cases_18_29,cases_30_39,cases_40_49,cases_50_59,cases_60_69,cases_70_79,cases_80
0,2020-01-25,Johor,4,4,0,4,0,4,0,0,...,0,0,0,0,0,1,0,0,0,0
1,2020-01-25,Kedah,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2020-01-25,Kelantan,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2020-01-25,Melaka,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2020-01-25,Negeri Sembilan,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
deaths_df = pd.read_csv("deaths_state.csv")
deaths_df.head()

,date,state,deaths_new,deaths_bid,deaths_new_dod,deaths_bid_dod,deaths_unvax,deaths_pvax,deaths_fvax,deaths_boost,deaths_tat
0,2020-03-17,Johor,1,0,1,0,1,0,0,0,0
1,2020-03-17,Kedah,0,0,0,0,0,0,0,0,0
2,2020-03-17,Kelantan,0,0,0,0,0,0,0,0,0
3,2020-03-17,Melaka,0,0,0,0,0,0,0,0,0
4,2020-03-17,Negeri Sembilan,0,0,0,0,0,0,0,0,0


In [4]:
beds_df = pd.read_csv("hospital.csv")
beds_df.head()

,date,state,beds,beds_covid,beds_noncrit,admitted_pui,admitted_covid,admitted_total,discharged_pui,discharged_covid,discharged_total,hosp_covid,hosp_pui,hosp_noncovid
0,2020-03-24,Johor,1440,434,1408,0,0,0,0,0,0,18,9,1
1,2020-03-24,Kedah,1218,143,1183,0,0,0,0,0,0,46,5,12
2,2020-03-24,Kelantan,1463,280,1424,9,6,15,1,0,1,78,58,23
3,2020-03-24,Melaka,1091,82,1065,0,0,0,0,0,0,19,10,12
4,2020-03-24,Negeri Sembilan,1223,253,1205,0,0,0,0,0,0,90,13,8


In [5]:
icu_df = pd.read_csv("icu.csv")
icu_df.head()

,date,state,beds_icu,beds_icu_rep,beds_icu_total,beds_icu_covid,vent,vent_port,icu_covid,icu_pui,icu_noncovid,vent_covid,vent_pui,vent_noncovid,vent_used,vent_port_used
0,2020-03-24,Johor,10,22,32,10,11,2,0,0,1,0,0,0,8,0
1,2020-03-24,Kedah,35,0,35,16,34,0,2,1,12,1,1,0,19,0
2,2020-03-24,Kelantan,39,0,39,20,42,4,1,0,20,1,0,0,26,0
3,2020-03-24,Melaka,26,0,26,12,23,4,0,2,12,0,1,0,13,5
4,2020-03-24,Negeri Sembilan,18,0,18,10,14,9,0,0,8,0,0,0,1,6


Drop extra columns and convert date to datetime

In [6]:
cases_df = cases_df.loc[:, cases_df.columns.intersection(['date','state', 'cases_new'])]
cases_df['date'] = pd.to_datetime(cases_df['date'])
cases_df.head()

,date,state,cases_new
0,2020-01-25,Johor,4
1,2020-01-25,Kedah,0
2,2020-01-25,Kelantan,0
3,2020-01-25,Melaka,0
4,2020-01-25,Negeri Sembilan,0


In [7]:
deaths_df = deaths_df.loc[:, deaths_df.columns.intersection(['date','state', 'deaths_new'])]
deaths_df['date'] = pd.to_datetime(deaths_df['date'])
deaths_df.head()

,date,state,deaths_new
0,2020-03-17,Johor,1
1,2020-03-17,Kedah,0
2,2020-03-17,Kelantan,0
3,2020-03-17,Melaka,0
4,2020-03-17,Negeri Sembilan,0


In [8]:
beds_df = pd.merge(beds_df, icu_df, on=['date','state'])
beds_df = beds_df.loc[:, beds_df.columns.intersection(['date','state', 'beds', 'beds_covid', 'beds_icu_covid'])]
beds_df['date'] = pd.to_datetime(beds_df['date'])
beds_df.head()

,date,state,beds,beds_covid,beds_icu_covid
0,2020-03-24,Johor,1440,434,10
1,2020-03-24,Kedah,1218,143,16
2,2020-03-24,Kelantan,1463,280,20
3,2020-03-24,Melaka,1091,82,12
4,2020-03-24,Negeri Sembilan,1223,253,10


# Combine datasets

In [9]:
from datetime import date, timedelta, datetime
# format: y/m/d
sdate = date(2020,1,1)
edate = date(2022,1,18)

states = list(cases_df['state'].unique())

new_df=pd.DataFrame(columns=['date','state', 'cases_new', 'deaths_new', 'beds', 'beds_covid', 'beds_icu_covid'])
row_count = 0
for date in pd.date_range(sdate,edate-timedelta(days=1),freq='d'):
    for state in states:
        row_count += 1
        new_df.loc[row_count, 'date'] = datetime.strftime(date, '%Y-%m-%d')
        new_df.loc[row_count, 'state'] = state
# new_df['date'] = pd.date_range(sdate,edate-timedelta(days=1),freq='d')
new_df.head(20)

,date,state,cases_new,deaths_new,beds,beds_covid,beds_icu_covid
1,2020-01-01,Johor,NaN,NaN,NaN,NaN,NaN
2,2020-01-01,Kedah,NaN,NaN,NaN,NaN,NaN
3,2020-01-01,Kelantan,NaN,NaN,NaN,NaN,NaN
4,2020-01-01,Melaka,NaN,NaN,NaN,NaN,NaN
5,2020-01-01,Negeri Sembilan,NaN,NaN,NaN,NaN,NaN
6,2020-01-01,Pahang,NaN,NaN,NaN,NaN,NaN
7,2020-01-01,Perak,NaN,NaN,NaN,NaN,NaN
8,2020-01-01,Perlis,NaN,NaN,NaN,NaN,NaN
9,2020-01-01,Pulau Pinang,NaN,NaN,NaN,NaN,NaN
10,2020-01-01,Sabah,NaN,NaN,NaN,NaN,NaN


Take data from old datasets into new datasets.

In [10]:
import datetime
def transpose(new_df, old_df, cols):
    for index, row in new_df.iterrows():
        current_date = new_df.loc[index, 'date']
        current_state = new_df.loc[index, 'state']
        for index1, row1 in old_df.loc[old_df['date'] == current_date].iterrows():
            state = old_df.loc[index1, 'state']
            if current_state == state:
                for col in cols:
                    new_df.loc[index, col] = old_df.loc[index1, col]

transpose(new_df, cases_df, ['cases_new'])
new_df.tail(20)

,date,state,cases_new,deaths_new,beds,beds_covid,beds_icu_covid
11949,2022-01-16,Terengganu,63,NaN,NaN,NaN,NaN
11950,2022-01-16,W.P. Kuala Lumpur,226,NaN,NaN,NaN,NaN
11951,2022-01-16,W.P. Labuan,14,NaN,NaN,NaN,NaN
11952,2022-01-16,W.P. Putrajaya,13,NaN,NaN,NaN,NaN
11953,2022-01-17,Johor,351,NaN,NaN,NaN,NaN
11954,2022-01-17,Kedah,268,NaN,NaN,NaN,NaN
11955,2022-01-17,Kelantan,172,NaN,NaN,NaN,NaN
11956,2022-01-17,Melaka,201,NaN,NaN,NaN,NaN
11957,2022-01-17,Negeri Sembilan,83,NaN,NaN,NaN,NaN
11958,2022-01-17,Pahang,127,NaN,NaN,NaN,NaN


In [11]:
transpose(new_df, deaths_df, ['deaths_new'])
new_df.tail(20)

,date,state,cases_new,deaths_new,beds,beds_covid,beds_icu_covid
11949,2022-01-16,Terengganu,63,0,NaN,NaN,NaN
11950,2022-01-16,W.P. Kuala Lumpur,226,0,NaN,NaN,NaN
11951,2022-01-16,W.P. Labuan,14,0,NaN,NaN,NaN
11952,2022-01-16,W.P. Putrajaya,13,0,NaN,NaN,NaN
11953,2022-01-17,Johor,351,1,NaN,NaN,NaN
11954,2022-01-17,Kedah,268,1,NaN,NaN,NaN
11955,2022-01-17,Kelantan,172,1,NaN,NaN,NaN
11956,2022-01-17,Melaka,201,0,NaN,NaN,NaN
11957,2022-01-17,Negeri Sembilan,83,2,NaN,NaN,NaN
11958,2022-01-17,Pahang,127,0,NaN,NaN,NaN


In [12]:
transpose(new_df, beds_df, ['beds', 'beds_covid', 'beds_icu_covid'])
new_df.tail(20)

,date,state,cases_new,deaths_new,beds,beds_covid,beds_icu_covid
11949,2022-01-16,Terengganu,63,0,1593,433,40
11950,2022-01-16,W.P. Kuala Lumpur,226,0,3963,572,34
11951,2022-01-16,W.P. Labuan,14,0,202,86,7
11952,2022-01-16,W.P. Putrajaya,13,0,609,44,6
11953,2022-01-17,Johor,351,1,5163,1858,46
11954,2022-01-17,Kedah,268,1,2664,1259,130
11955,2022-01-17,Kelantan,172,1,2495,368,48
11956,2022-01-17,Melaka,201,0,1317,651,24
11957,2022-01-17,Negeri Sembilan,83,2,1749,1084,68
11958,2022-01-17,Pahang,127,0,2002,659,81


In [13]:
new_df.dtypes

date              object
state             object
cases_new         object
deaths_new        object
beds              object
beds_covid        object
beds_icu_covid    object
dtype: object

In [14]:
new_df[["cases_new", "deaths_new", "beds", "beds_covid", "beds_icu_covid"]] = new_df[["cases_new", "deaths_new", "beds", "beds_covid", "beds_icu_covid"]].apply(pd.to_numeric)
new_df.dtypes

date               object
state              object
cases_new         float64
deaths_new        float64
beds              float64
beds_covid        float64
beds_icu_covid    float64
dtype: object

In [15]:
new_df=new_df.sort_values(['date']).reset_index(drop=True)
new_df["cum_cases"]=new_df.groupby(['state'])['cases_new'].cumsum(axis=0)
new_df["cum_deaths"]=new_df.groupby(['state'])['deaths_new'].cumsum(axis=0)
new_df.sort_values(['date', 'state'])
new_df.tail(20)

,date,state,cases_new,deaths_new,beds,beds_covid,beds_icu_covid,cum_cases,cum_deaths
11948,2022-01-16,Kelantan,160.0,1.0,2495.0,368.0,48.0,173557.0,1266.0
11949,2022-01-16,Kedah,175.0,2.0,2664.0,1259.0,130.0,171828.0,2179.0
11950,2022-01-16,Johor,371.0,1.0,5163.0,1858.0,46.0,250780.0,3893.0
11951,2022-01-16,Perlis,3.0,0.0,503.0,156.0,11.0,7405.0,137.0
11952,2022-01-17,W.P. Kuala Lumpur,168.0,0.0,3963.0,572.0,34.0,214997.0,2692.0
11953,2022-01-17,Terengganu,64.0,3.0,1593.0,433.0,40.0,84354.0,746.0
11954,2022-01-17,Selangor,524.0,5.0,5338.0,1166.0,78.0,800311.0,10021.0
11955,2022-01-17,Sarawak,9.0,0.0,3569.0,756.0,91.0,252609.0,1620.0
11956,2022-01-17,Sabah,163.0,1.0,4270.0,1108.0,120.0,244380.0,2809.0
11957,2022-01-17,Pulau Pinang,84.0,2.0,1978.0,510.0,49.0,164249.0,1763.0


In [16]:
new_df['state'] = new_df['state'].replace(['Pulau Pinang'],'Penang')
new_df['state'] = new_df['state'].replace(['W.P. Kuala Lumpur'],'Kuala Lumpur')
new_df['state'] = new_df['state'].replace(['W.P. Labuan'],'Labuan')
new_df['state'] = new_df['state'].replace(['W.P. Putrajaya'],'Putrajaya')
new_df['state'] = new_df['state'].replace(['Melaka'],'Malacca')
new_df.tail(20)

,date,state,cases_new,deaths_new,beds,beds_covid,beds_icu_covid,cum_cases,cum_deaths
11948,2022-01-16,Kelantan,160.0,1.0,2495.0,368.0,48.0,173557.0,1266.0
11949,2022-01-16,Kedah,175.0,2.0,2664.0,1259.0,130.0,171828.0,2179.0
11950,2022-01-16,Johor,371.0,1.0,5163.0,1858.0,46.0,250780.0,3893.0
11951,2022-01-16,Perlis,3.0,0.0,503.0,156.0,11.0,7405.0,137.0
11952,2022-01-17,Kuala Lumpur,168.0,0.0,3963.0,572.0,34.0,214997.0,2692.0
11953,2022-01-17,Terengganu,64.0,3.0,1593.0,433.0,40.0,84354.0,746.0
11954,2022-01-17,Selangor,524.0,5.0,5338.0,1166.0,78.0,800311.0,10021.0
11955,2022-01-17,Sarawak,9.0,0.0,3569.0,756.0,91.0,252609.0,1620.0
11956,2022-01-17,Sabah,163.0,1.0,4270.0,1108.0,120.0,244380.0,2809.0
11957,2022-01-17,Penang,84.0,2.0,1978.0,510.0,49.0,164249.0,1763.0


In [17]:
new_df.to_csv("daily.csv", index=False)